In [137]:
import tensorflow as tf
import numpy as np
import csv

In [138]:
# Load the data from the CSV file
with open('data.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    data = []
    for row in reader:
        data.append(row)

In [139]:
print(len(data)) 
print(len(data[0]))

19
2


In [140]:
# Extract the questions and answers from the data
questions = [row[0] for row in data]
answers = [row[1] for row in data]

In [141]:
# Build a vocabulary from the questions and answers
vocab = set()
for q, a in zip(questions, answers):
    vocab |= set(q.lower().split())
    vocab |= set(a.lower().split())
vocab = sorted(list(vocab))

In [142]:

vocab_size = len(vocab)
seq_len = 15  # set a fixed sequence length
question_seqs = np.zeros((len(questions), seq_len), dtype=np.int32)
answer_seqs = np.zeros((len(answers), seq_len), dtype=np.int32)
for i, (q, a) in enumerate(zip(questions, answers)):
    q_seq = [vocab.index(w.lower()) for w in q.split()]
    a_seq = [vocab.index(w.lower()) for w in a.split()]
    q_seq = q_seq[:seq_len]  # truncate questions longer than seq_len
    a_seq = a_seq[:seq_len]  # truncate answers longer than seq_len
    question_seqs[i, :len(q_seq)] = q_seq
    answer_seqs[i, :len(a_seq)] = a_seq

In [143]:
# Build the model
embedding_dim = 128
rnn_units = 256
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])

In [144]:
# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))


In [145]:
# Train the model
model.fit(question_seqs, answer_seqs, epochs=500)

Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 4.9750
Epoch 2/500
1/1 [==============================] - 0s 19ms/step - loss: 4.9389
Epoch 3/500
1/1 [==============================] - 0s 20ms/step - loss: 4.9015
Epoch 4/500
1/1 [==============================] - 0s 23ms/step - loss: 4.8576
Epoch 5/500
1/1 [==============================] - 0s 18ms/step - loss: 4.8004
Epoch 6/500
1/1 [==============================] - 0s 20ms/step - loss: 4.7200
Epoch 7/500
1/1 [==============================] - 0s 20ms/step - loss: 4.5998
Epoch 8/500
1/1 [==============================] - 0s 21ms/step - loss: 4.4104
Epoch 9/500
1/1 [==============================] - 0s 18ms/step - loss: 4.1090
Epoch 10/500
1/1 [==============================] - 0s 18ms/step - loss: 3.7015
Epoch 11/500
1/1 [==============================] - 0s 20ms/step - loss: 3.3666
Epoch 12/500
1/1 [==============================] - 0s 19ms/step - loss: 3.2301
Epoch 13/500
1/1 [==============================] -

1/1 [==============================] - 0s 16ms/step - loss: 2.2399
Epoch 104/500
1/1 [==============================] - 0s 18ms/step - loss: 2.2257
Epoch 105/500
1/1 [==============================] - 0s 17ms/step - loss: 2.2115
Epoch 106/500
1/1 [==============================] - 0s 17ms/step - loss: 2.1971
Epoch 107/500
1/1 [==============================] - 0s 17ms/step - loss: 2.1826
Epoch 108/500
1/1 [==============================] - 0s 17ms/step - loss: 2.1679
Epoch 109/500
1/1 [==============================] - 0s 16ms/step - loss: 2.1529
Epoch 110/500
1/1 [==============================] - 0s 19ms/step - loss: 2.1376
Epoch 111/500
1/1 [==============================] - 0s 21ms/step - loss: 2.1220
Epoch 112/500
1/1 [==============================] - 0s 18ms/step - loss: 2.1063
Epoch 113/500
1/1 [==============================] - 0s 19ms/step - loss: 2.0915
Epoch 114/500
1/1 [==============================] - 0s 18ms/step - loss: 2.0812
Epoch 115/500
1/1 [=======================

1/1 [==============================] - 0s 16ms/step - loss: 0.7724
Epoch 205/500
1/1 [==============================] - 0s 18ms/step - loss: 0.7622
Epoch 206/500
1/1 [==============================] - 0s 19ms/step - loss: 0.7500
Epoch 207/500
1/1 [==============================] - 0s 17ms/step - loss: 0.7361
Epoch 208/500
1/1 [==============================] - 0s 17ms/step - loss: 0.7233
Epoch 209/500
1/1 [==============================] - 0s 17ms/step - loss: 0.7133
Epoch 210/500
1/1 [==============================] - 0s 19ms/step - loss: 0.7052
Epoch 211/500
1/1 [==============================] - 0s 18ms/step - loss: 0.6977
Epoch 212/500
1/1 [==============================] - 0s 16ms/step - loss: 0.6897
Epoch 213/500
1/1 [==============================] - 0s 19ms/step - loss: 0.6801
Epoch 214/500
1/1 [==============================] - 0s 21ms/step - loss: 0.6673
Epoch 215/500
1/1 [==============================] - 0s 19ms/step - loss: 0.6538
Epoch 216/500
1/1 [=======================

1/1 [==============================] - 0s 17ms/step - loss: 0.2052
Epoch 306/500
1/1 [==============================] - 0s 18ms/step - loss: 0.2037
Epoch 307/500
1/1 [==============================] - 0s 18ms/step - loss: 0.2022
Epoch 308/500
1/1 [==============================] - 0s 20ms/step - loss: 0.2007
Epoch 309/500
1/1 [==============================] - 0s 19ms/step - loss: 0.1993
Epoch 310/500
1/1 [==============================] - 0s 19ms/step - loss: 0.1979
Epoch 311/500
1/1 [==============================] - 0s 18ms/step - loss: 0.1967
Epoch 312/500
1/1 [==============================] - 0s 18ms/step - loss: 0.1954
Epoch 313/500
1/1 [==============================] - 0s 18ms/step - loss: 0.1942
Epoch 314/500
1/1 [==============================] - 0s 18ms/step - loss: 0.1930
Epoch 315/500
1/1 [==============================] - 0s 19ms/step - loss: 0.1919
Epoch 316/500
1/1 [==============================] - 0s 20ms/step - loss: 0.1907
Epoch 317/500
1/1 [=======================

1/1 [==============================] - 0s 18ms/step - loss: 0.1408
Epoch 407/500
1/1 [==============================] - 0s 20ms/step - loss: 0.1406
Epoch 408/500
1/1 [==============================] - 0s 17ms/step - loss: 0.1403
Epoch 409/500
1/1 [==============================] - 0s 16ms/step - loss: 0.1401
Epoch 410/500
1/1 [==============================] - 0s 17ms/step - loss: 0.1398
Epoch 411/500
1/1 [==============================] - 0s 18ms/step - loss: 0.1396
Epoch 412/500
1/1 [==============================] - 0s 16ms/step - loss: 0.1393
Epoch 413/500
1/1 [==============================] - 0s 17ms/step - loss: 0.1391
Epoch 414/500
1/1 [==============================] - 0s 17ms/step - loss: 0.1389
Epoch 415/500
1/1 [==============================] - 0s 18ms/step - loss: 0.1386
Epoch 416/500
1/1 [==============================] - 0s 19ms/step - loss: 0.1384
Epoch 417/500
1/1 [==============================] - 0s 19ms/step - loss: 0.1382
Epoch 418/500
1/1 [=======================

In [146]:
# Define a function to generate responses
def generate_response(model, question, vocab):
    question_seq = np.zeros((1, seq_len), dtype=np.int32)
    q_seq = [vocab.index(w.lower()) for w in question.split()]
    question_seq[0, :len(q_seq)] = q_seq
    logits = model.predict(question_seq)
    response_seq = tf.argmax(logits, axis=-1)[0]
    response = ' '.join([vocab[i] for i in response_seq if i != 0])
    return response

In [148]:
 Test the model
while True:
    question = input('You: ')
    if question == 'exit':
        break
    response = generate_response(model, question, vocab)
    print('Bot:', response)